In [1]:
from transformers import LiltForPretraining, HarrysConfig, HarrysInvConfig
from pl_lilt_datamodule import LiltPretrainingDataModule
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser = LiltPretrainingDataModule.add_model_specific_args(parser)
args = parser.parse_args('')

dm = LiltPretrainingDataModule(args)
dm.prepare_data()
dm.setup()
train_loader = dm.train_dataloader()
batch = next(iter(train_loader))

Found cached dataset rvl_cdip (/home/hj36wegi/scratch/data/huggingface/datasets/rvl_cdip/default/1.0.0/ea410993ed3f5b9744d8616ffbaad5f70a75a21a4233626dd07b3de31d381e53)


  0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: stack expects each tensor to be equal size, but got [1000, 777] at entry 0 and [1000, 769] at entry 1

In [ ]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'bbox', 'bbox_loc', 'input_labels', 'bbox_labels'])

In [3]:
lang_adapter_config = HarrysInvConfig(text_output_adapter=True, layout_output_adapter=False)
layout_adapter_config = HarrysConfig(text_output_adapter=False, layout_output_adapter=True)
task_adapter_config = HarrysConfig(text_output_adapter=True, layout_output_adapter=False)

In [4]:
model = LiltForPretraining.from_pretrained("nielsr/lilt-xlm-roberta-base")

Some weights of LiltForPretraining were not initialized from the model checkpoint at nielsr/lilt-xlm-roberta-base and are newly initialized: ['lm_head.kpl_head.layer_norm.bias', 'lm_head.mvlm_head.dense.weight', 'lm_head.mvlm_head.layer_norm.bias', 'lm_head.cai_head.decoder.weight', 'lm_head.cai_head.dense.bias', 'lm_head.cai_head.layer_norm.weight', 'lm_head.cai_head.decoder.bias', 'lm_head.kpl_head.layer_norm.weight', 'lm_head.mvlm_head.layer_norm.weight', 'lm_head.cai_head.dense.weight', 'lm_head.kpl_head.dense.weight', 'lm_head.kpl_head.decoder.weight', 'lm_head.mvlm_head.bias', 'lm_head.mvlm_head.dense.bias', 'lm_head.kpl_head.bias', 'lm_head.kpl_head.decoder.bias', 'lm_head.kpl_head.dense.bias', 'lm_head.cai_head.bias', 'lm_head.cai_head.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model.add_adapter("layout", layout_adapter_config)
model.add_adapter("language", lang_adapter_config)
model.add_adapter("task", task_adapter_config)

In [8]:
model.train_adapter?

Signature:
model.train_adapter(
    adapter_setup: Union[list, transformers.adapters.composition.AdapterCompositionBlock],
    train_embeddings=False,
)
Docstring:
Sets the model into mode for training the given adapters. If self.base_model is self, must inherit from a class
that implements this method, to preclude infinite recursion
File:      ~/.conda/envs/adapters/lib/python3.10/site-packages/transformers/adapters/model_mixin.py
Type:      method

In [6]:
amodel.train_adapter(["layout","task", "language"])
# model.freeze_model(False)


In [7]:
model.lilt.encoder.layer[0]

LiltLayer(
  (attention): LiltAttention(
    (self): LiltSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (layout_query): Linear(in_features=192, out_features=192, bias=True)
      (layout_key): Linear(in_features=192, out_features=192, bias=True)
      (layout_value): Linear(in_features=192, out_features=192, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): LiltSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (adapters): ModuleDict()
      (adapter_fusion_layer): ModuleDict()
    )
    (layout_output): LiltSelfOutput(
      (dense): Linear(in_features=192, out_features=192, bias=True)
      (LayerNorm): LayerNorm((192,), eps=1e-05,

In [21]:
out = model(**batch)

language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout


In [22]:
for n,p in model.lilt.encoder.layer[0].named_parameters():
    if 'adapter' in n:
        if p.requires_grad:
            print("+ " + n)
        else:
            print("- " + n)

+ output.adapters.language.adapter_down.0.weight
+ output.adapters.language.adapter_down.0.bias
+ output.adapters.language.adapter_up.weight
+ output.adapters.language.adapter_up.bias
+ output.adapters.task.adapter_down.0.weight
+ output.adapters.task.adapter_down.0.bias
+ output.adapters.task.adapter_up.weight
+ output.adapters.task.adapter_up.bias
+ layout_output.adapters.layout.adapter_down.0.weight
+ layout_output.adapters.layout.adapter_down.0.bias
+ layout_output.adapters.layout.adapter_up.weight
+ layout_output.adapters.layout.adapter_up.bias


In [23]:
p

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)